# Set Up

In [24]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns= 99999


In [25]:
!pip install meetup-api

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Pull Data from Meet Up

In [26]:
import meetup.api
# Get key from https://secure.meetup.com/meetup_api/key/
my_special_api_key_value = "put key here"
client = meetup.api.Client(my_special_api_key_value)


In [27]:
tech_meetup_category = "34" # Tech Meetups see ids in the docs
ecosystem = "Israel" # Defaults to Israel 
# This code enables batching of all relevant results not just the first page
groups = []
groups_per_page = 200
i=0
while True:
    find_groups = client.GetFindGroups(country=ecosystem, category=tech_meetup_category,
                                       ordering="most_active", pages=groups_per_page, offset=i)
    if len(find_groups) <= 0:
        break
    else:
        groups += find_groups
    i+=1

29/30 (10 seconds remaining)
28/30 (9 seconds remaining)
27/30 (7 seconds remaining)
26/30 (7 seconds remaining)


# Process Data For Analysis 

### Turn data into pandas data frame

In [28]:
groups_df = pd.DataFrame([g.__dict__ for g in groups]).sort_values(['members', 'status'], ascending=[0, 0])
groups_df.head(5)

,category,city,country,created,description,group_photo,id,join_mode,key_photo,lat,link,localized_country_name,localized_location,lon,members,meta_category,name,next_event,organizer,pro_network,score,state,status,timezone,untranslated_city,urlname,visibility,who
9,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1328085870000,"<p>This is a community for <a href=""https://ww...","{'id': 469196353, 'highres_link': 'https://sec...",3212532,open,"{'id': 462383898, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/full-stack-developer-il/,Israel,"Tel Aviv-Yafo, Israel",34.77,10295,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Fullstack Developers Israel,"{'id': '252902953', 'name': 'Use micro fronten...","{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,full-stack-developer-il,public,Fullstackers
557,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1273648309000,<p>Impact Hub Tel Aviv - TECH on BEER is a mee...,"{'id': 458540400, 'highres_link': 'https://sec...",1644280,open,"{'id': 446555822, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Tel-Aviv-Nightowls/,Israel,"Tel Aviv-Yafo, Israel",34.77,9257,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",DIGI Hub Tel Aviv - TECH on BEER,NaN,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Tel-Aviv-Nightowls,public,Techies
6,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1387171943000,"<p>At <a href=""http://wix.engineering"">Wix&nbs...","{'id': 471533117, 'highres_link': 'https://sec...",11468592,open,"{'id': 471532924, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/at-wix/,Israel,"Tel Aviv-Yafo, Israel",34.77,9100,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Meetups at Wix,"{'id': '255190955', 'name': 'SSR for Performan...","{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,at-wix,public,Members
5,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1321804282000,<p>Product Management Meetup (IL) has been set...,"{'id': 452118584, 'highres_link': 'https://sec...",2841102,open,"{'id': 452118535, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/PMM-IL/,Israel,"Tel Aviv-Yafo, Israel",34.77,8476,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Product Management Meetup (IL),"{'id': '255545705', 'name': 'Building a startu...","{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,PMM-IL,public,Members
159,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1301054021000,<p>The group is meant to be a hub for those in...,"{'id': 407475712, 'highres_link': 'https://sec...",1804355,open,"{'id': 439002726, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Big-Data-Israel/,Israel,"Tel Aviv-Yafo, Israel",34.77,8068,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Big Data & Data Science - Israel,NaN,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Big-Data-Israel,public,Big Data Professionals


### Clean HTML from Description text

In [29]:
from bs4 import BeautifulSoup
def clean_text(text):
    if text:
        t = BeautifulSoup(text, "lxml").text
    #    remove all tabs and newlines needed for tsv
        return ' '.join(t.split())
    return ''

In [30]:
groups_df.description = groups_df.description.apply(lambda x :clean_text(x))
groups_df.head(5)

,category,city,country,created,description,group_photo,id,join_mode,key_photo,lat,link,localized_country_name,localized_location,lon,members,meta_category,name,next_event,organizer,pro_network,score,state,status,timezone,untranslated_city,urlname,visibility,who
9,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1328085870000,This is a community for fullstack developers w...,"{'id': 469196353, 'highres_link': 'https://sec...",3212532,open,"{'id': 462383898, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/full-stack-developer-il/,Israel,"Tel Aviv-Yafo, Israel",34.77,10295,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Fullstack Developers Israel,"{'id': '252902953', 'name': 'Use micro fronten...","{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,full-stack-developer-il,public,Fullstackers
557,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1273648309000,Impact Hub Tel Aviv - TECH on BEER is a meetup...,"{'id': 458540400, 'highres_link': 'https://sec...",1644280,open,"{'id': 446555822, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Tel-Aviv-Nightowls/,Israel,"Tel Aviv-Yafo, Israel",34.77,9257,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",DIGI Hub Tel Aviv - TECH on BEER,NaN,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Tel-Aviv-Nightowls,public,Techies
6,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1387171943000,At Wix Engineering we have internal tech talks...,"{'id': 471533117, 'highres_link': 'https://sec...",11468592,open,"{'id': 471532924, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/at-wix/,Israel,"Tel Aviv-Yafo, Israel",34.77,9100,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Meetups at Wix,"{'id': '255190955', 'name': 'SSR for Performan...","{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,at-wix,public,Members
5,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1321804282000,Product Management Meetup (IL) has been set up...,"{'id': 452118584, 'highres_link': 'https://sec...",2841102,open,"{'id': 452118535, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/PMM-IL/,Israel,"Tel Aviv-Yafo, Israel",34.77,8476,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Product Management Meetup (IL),"{'id': '255545705', 'name': 'Building a startu...","{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,PMM-IL,public,Members
159,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,1301054021000,The group is meant to be a hub for those invol...,"{'id': 407475712, 'highres_link': 'https://sec...",1804355,open,"{'id': 439002726, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Big-Data-Israel/,Israel,"Tel Aviv-Yafo, Israel",34.77,8068,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Big Data & Data Science - Israel,NaN,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Big-Data-Israel,public,Big Data Professionals


### Clean Convert Time from UTC to Date Time

In [31]:
groups_df.created = pd.to_datetime(groups_df.created, unit='ms')
groups_df.head(5)

,category,city,country,created,description,group_photo,id,join_mode,key_photo,lat,link,localized_country_name,localized_location,lon,members,meta_category,name,next_event,organizer,pro_network,score,state,status,timezone,untranslated_city,urlname,visibility,who
9,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,2012-02-01 08:44:30,This is a community for fullstack developers w...,"{'id': 469196353, 'highres_link': 'https://sec...",3212532,open,"{'id': 462383898, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/full-stack-developer-il/,Israel,"Tel Aviv-Yafo, Israel",34.77,10295,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Fullstack Developers Israel,"{'id': '252902953', 'name': 'Use micro fronten...","{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,full-stack-developer-il,public,Fullstackers
557,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,2010-05-12 07:11:49,Impact Hub Tel Aviv - TECH on BEER is a meetup...,"{'id': 458540400, 'highres_link': 'https://sec...",1644280,open,"{'id': 446555822, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Tel-Aviv-Nightowls/,Israel,"Tel Aviv-Yafo, Israel",34.77,9257,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",DIGI Hub Tel Aviv - TECH on BEER,NaN,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Tel-Aviv-Nightowls,public,Techies
6,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,2013-12-16 05:32:23,At Wix Engineering we have internal tech talks...,"{'id': 471533117, 'highres_link': 'https://sec...",11468592,open,"{'id': 471532924, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/at-wix/,Israel,"Tel Aviv-Yafo, Israel",34.77,9100,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Meetups at Wix,"{'id': '255190955', 'name': 'SSR for Performan...","{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,at-wix,public,Members
5,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,2011-11-20 15:51:22,Product Management Meetup (IL) has been set up...,"{'id': 452118584, 'highres_link': 'https://sec...",2841102,open,"{'id': 452118535, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/PMM-IL/,Israel,"Tel Aviv-Yafo, Israel",34.77,8476,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Product Management Meetup (IL),"{'id': '255545705', 'name': 'Building a startu...","{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,PMM-IL,public,Members
159,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",Tel Aviv-Yafo,IL,2011-03-25 11:53:41,The group is meant to be a hub for those invol...,"{'id': 407475712, 'highres_link': 'https://sec...",1804355,open,"{'id': 439002726, 'highres_link': 'https://sec...",32.07,https://www.meetup.com/Big-Data-Israel/,Israel,"Tel Aviv-Yafo, Israel",34.77,8068,"{'id': 292, 'shortname': 'tech', 'name': 'Tech...",Big Data & Data Science - Israel,NaN,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",NaN,1.0,,active,Asia/Jerusalem,Tel Aviv-Yafo,Big-Data-Israel,public,Big Data Professionals


# Analysis 

### How many relevant groups are there?

In [32]:
"{} groups found".format(len(groups))

'566 groups found'

### What information about the groups can I query ?

In [33]:
groups_df.columns

Index(['category', 'city', 'country', 'created', 'description', 'group_photo',
       'id', 'join_mode', 'key_photo', 'lat', 'link', 'localized_country_name',
       'localized_location', 'lon', 'members', 'meta_category', 'name',
       'next_event', 'organizer', 'pro_network', 'score', 'state', 'status',
       'timezone', 'untranslated_city', 'urlname', 'visibility', 'who'],
      dtype='object')

### Lets filter this

In [34]:
ecosystem_df = groups_df[['name', 'members', 'organizer', 'status', 'created', 'who', 'description']]
ecosystem_df.head(5)

,name,members,organizer,status,created,who,description
9,Fullstack Developers Israel,10295,"{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",active,2012-02-01 08:44:30,Fullstackers,This is a community for fullstack developers w...
557,DIGI Hub Tel Aviv - TECH on BEER,9257,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",active,2010-05-12 07:11:49,Techies,Impact Hub Tel Aviv - TECH on BEER is a meetup...
6,Meetups at Wix,9100,"{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",active,2013-12-16 05:32:23,Members,At Wix Engineering we have internal tech talks...
5,Product Management Meetup (IL),8476,"{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",active,2011-11-20 15:51:22,Members,Product Management Meetup (IL) has been set up...
159,Big Data & Data Science - Israel,8068,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",active,2011-03-25 11:53:41,Big Data Professionals,The group is meant to be a hub for those invol...


### For each group lets find when their next event will be and what it will be called

In [35]:
ecosystem_df["next_event_date"] = pd.to_datetime(groups_df.next_event.apply(lambda x : x["time"] if not x != x else "") , unit='ms')
ecosystem_df.head(5)

C:\Users\abornst\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,name,members,organizer,status,created,who,description,next_event_date
9,Fullstack Developers Israel,10295,"{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",active,2012-02-01 08:44:30,Fullstackers,This is a community for fullstack developers w...,2018-10-28 15:30:00
557,DIGI Hub Tel Aviv - TECH on BEER,9257,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",active,2010-05-12 07:11:49,Techies,Impact Hub Tel Aviv - TECH on BEER is a meetup...,NaT
6,Meetups at Wix,9100,"{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",active,2013-12-16 05:32:23,Members,At Wix Engineering we have internal tech talks...,2018-10-24 15:00:00
5,Product Management Meetup (IL),8476,"{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",active,2011-11-20 15:51:22,Members,Product Management Meetup (IL) has been set up...,2018-11-01 16:30:00
159,Big Data & Data Science - Israel,8068,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",active,2011-03-25 11:53:41,Big Data Professionals,The group is meant to be a hub for those invol...,NaT


In [36]:
ecosystem_df["next_event_name"] = groups_df.next_event.apply(lambda x : x["name"] if not x != x else "")
ecosystem_df.head(5)

C:\Users\abornst\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,name,members,organizer,status,created,who,description,next_event_date,next_event_name
9,Fullstack Developers Israel,10295,"{'id': 2537432, 'name': 'Lior Kanfi', 'bio': '...",active,2012-02-01 08:44:30,Fullstackers,This is a community for fullstack developers w...,2018-10-28 15:30:00,Use micro frontends approach to work with lega...
557,DIGI Hub Tel Aviv - TECH on BEER,9257,"{'id': 114781752, 'name': 'Yoram', 'bio': '', ...",active,2010-05-12 07:11:49,Techies,Impact Hub Tel Aviv - TECH on BEER is a meetup...,NaT,
6,Meetups at Wix,9100,"{'id': 14426994, 'name': 'Aviran Mordo', 'bio'...",active,2013-12-16 05:32:23,Members,At Wix Engineering we have internal tech talks...,2018-10-24 15:00:00,SSR for Performance and SEO and Distinguishing...
5,Product Management Meetup (IL),8476,"{'id': 10306298, 'name': 'Osnat Niv-Assa', 'bi...",active,2011-11-20 15:51:22,Members,Product Management Meetup (IL) has been set up...,2018-11-01 16:30:00,Building a startup inside your organization: T...
159,Big Data & Data Science - Israel,8068,"{'id': 13504754, 'name': 'Asaf - Birenzvieg Ho...",active,2011-03-25 11:53:41,Big Data Professionals,The group is meant to be a hub for those invol...,NaT,


### For each group lets get the primary orginizers

In [39]:
ecosystem_df["organizer"] = ecosystem_df.organizer.apply(lambda x : x["name"])
ecosystem_df.head(5)

C:\Users\abornst\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,name,members,organizer,status,created,who,description,next_event_date,next_event_name
9,Fullstack Developers Israel,10295,Lior Kanfi,active,2012-02-01 08:44:30,Fullstackers,This is a community for fullstack developers w...,2018-10-28 15:30:00,Use micro frontends approach to work with lega...
557,DIGI Hub Tel Aviv - TECH on BEER,9257,Yoram,active,2010-05-12 07:11:49,Techies,Impact Hub Tel Aviv - TECH on BEER is a meetup...,NaT,
6,Meetups at Wix,9100,Aviran Mordo,active,2013-12-16 05:32:23,Members,At Wix Engineering we have internal tech talks...,2018-10-24 15:00:00,SSR for Performance and SEO and Distinguishing...
5,Product Management Meetup (IL),8476,Osnat Niv-Assa,active,2011-11-20 15:51:22,Members,Product Management Meetup (IL) has been set up...,2018-11-01 16:30:00,Building a startup inside your organization: T...
159,Big Data & Data Science - Israel,8068,Asaf - Birenzvieg Holdings Ltd.,active,2011-03-25 11:53:41,Big Data Professionals,The group is meant to be a hub for those invol...,NaT,


### Lets see all groups with more than 1000 members

In [40]:
ecosystem_df[ecosystem_df["members"]> 1000]

,name,members,organizer,status,created,who,description,next_event_date,next_event_name
9,Fullstack Developers Israel,10295,Lior Kanfi,active,2012-02-01 08:44:30,Fullstackers,This is a community for fullstack developers w...,2018-10-28 15:30:00,Use micro frontends approach to work with lega...
557,DIGI Hub Tel Aviv - TECH on BEER,9257,Yoram,active,2010-05-12 07:11:49,Techies,Impact Hub Tel Aviv - TECH on BEER is a meetup...,NaT,
6,Meetups at Wix,9100,Aviran Mordo,active,2013-12-16 05:32:23,Members,At Wix Engineering we have internal tech talks...,2018-10-24 15:00:00,SSR for Performance and SEO and Distinguishing...
5,Product Management Meetup (IL),8476,Osnat Niv-Assa,active,2011-11-20 15:51:22,Members,Product Management Meetup (IL) has been set up...,2018-11-01 16:30:00,Building a startup inside your organization: T...
159,Big Data & Data Science - Israel,8068,Asaf - Birenzvieg Holdings Ltd.,active,2011-03-25 11:53:41,Big Data Professionals,The group is meant to be a hub for those invol...,NaT,
56,8200 EISP - Entrepreneurship and Innovation Su...,7818,Mor Chen,active,2012-10-28 09:30:12,Entrepreneurs,Welcome entrepreneurs! This group is for anyon...,2018-10-29 16:00:00,Things I've learned in my first year as an ent...
137,she codes(tlv);,7416,Ruth Polachek,active,2014-06-14 11:44:22,Coders,;she codes הינה קהילת נשים מתכנתות. הארגון שם ...,NaT,
91,IGTCloud,6637,Avner Algom,active,2010-10-18 07:57:29,Members,The IGTCloud is a non-profit organization of l...,2018-11-05 07:00:00,Cloud Native Israel 2018
254,JavaScript Israel,6329,Shai Reznik,active,2013-01-07 16:13:24,JavaScripters,Visit our website - http://www.jsisrael.com/ A...,NaT,
15,Israel AWS User Group,6247,Arthur Schmunk,active,2013-02-26 11:20:10,Members,The Israel chapter of Amazon Web Services User...,NaT,


### Lets see who the primary orginizers are for these groups so we can see if they are interesting

In [41]:
oragnizers_df = pd.DataFrame(list(groups_df['organizer']))

In [43]:
oragnizers_df.bio = oragnizers_df.bio.apply(lambda x:  ' '.join(x.split()))

In [44]:
oragnizers_df[['name','bio','id']]

,name,bio,id
0,Lior Kanfi,,2537432
1,Yoram,,114781752
2,Aviran Mordo,Head of engineering @Wix,14426994
3,Osnat Niv-Assa,,10306298
4,Asaf - Birenzvieg Holdings Ltd.,Entrepreneur | Startup Advisor | Engineer | Ex...,13504754
5,Mor Chen,,260573334
6,Ruth Polachek,,150659252
7,Avner Algom,,7993785
8,Shai Reznik,Founder & Instructor of http://www.hirez.io An...,37252682
9,Arthur Schmunk,,35336252


### Lets see what the Upcomming Events Are

In [45]:
upcomming_events = pd.DataFrame([x for x in groups_df['next_event'] if str(x) != 'nan'])

In [46]:
upcomming_events.name =  upcomming_events.name.apply(lambda x:  ' '.join(x.split()))

In [47]:
upcomming_events.time = pd.to_datetime(upcomming_events.time, unit='ms')

In [49]:
upcomming_events

,id,name,time,utc_offset,yes_rsvp_count
0,252902953,Use micro frontends approach to work with lega...,2018-10-28 15:30:00,7200000,100
1,255190955,SSR for Performance and SEO and Distinguishing...,2018-10-24 15:00:00,10800000,130
2,255545705,Building a startup inside your organization: T...,2018-11-01 16:30:00,7200000,120
3,255737686,Things I've learned in my first year as an ent...,2018-10-29 16:00:00,7200000,28
4,251876853,Cloud Native Israel 2018,2018-11-05 07:00:00,7200000,40
5,254373002,Designing the Customer Experience (3 Talks),2018-11-21 16:00:00,7200000,1
6,251794304,Startups meet Capital,2018-11-20 16:00:00,7200000,76
7,255521836,Streaming things with Kafka and Spark,2018-10-31 16:00:00,7200000,208
8,255743911,BE MASTER #1 - Frontend MasterClass by 500Tech...,2018-11-12 16:30:00,7200000,67
9,253545071,PyData #17 - Sponsored by SimilarWeb,2018-11-07 16:00:00,7200000,150


### Lets export this to excel

In [13]:
groups_df.to_csv(path_or_buf='israel_groups.tsv', sep='\t',)

In [17]:
oragnizers_df[['name','bio','id']].to_csv(path_or_buf='israel_orginizers.tsv', sep='\t')

In [21]:
upcomming_events[['id','name','time','yes_rsvp_count']].to_csv(path_or_buf='upcoming_events.tsv', sep='\t')